# 크롤러 테스트중

In [94]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
HEADERS = {'User-Agent' : user_agent}

In [95]:
def get_chosunbiz_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # #fusion-app > div.article > div:nth-child(2) > div > section > article
    # #fusion-app > div.article > div:nth-child(2) > div > section > article > section
    contents = soup.select_one('#fusion-app > div.article > div:nth-child(2) > div > section > article > section')
    content = ''
    print(contents)
    p_tags = contents.select('p')
    for p in p_tags:
        content += p.text
    return content

In [96]:
def get_nocut_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # #pnlContent
    content = soup.select_one('div#pnlContent').text
    return link, content

In [97]:
def get_etnews_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # body > section > section > article
    contents = soup.select_one('body > section > section > article')
    content = ''
    p_tags = contents.select('p')
    for p in p_tags:
        content += p.text
    return link, content

In [98]:
def get_hankyung_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # #articletxt
    # #articletxt
    content = soup.find('div', {'id':'articletxt'}).text
    return link, content

In [99]:
def get_joongang_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # #article_body
    contents = soup.select_one('div#article_body')
    content = ''
    p_tags = contents.select('p')
    for p in p_tags:
        content += p.text
    return link, content

In [100]:
def get_moneytoday_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # #textBody
    content = soup.select_one('div#textBody').text
    return link, content

In [116]:
def get_newsone_content(link, headers=HEADERS):
    r = requests.get(link)
    soup = bs(r.text, 'lxml')
    # #articles_detail
    content = soup.select_one('div#articles_detail').text
    return link, content

In [131]:
def get_hankookilbo_content(link):
    r = requests.get(link, headers=HEADERS)
    soup = bs(r.text, 'lxml')
    # body > div.wrap.imp-end > div.container.end-uni > div.end-body > div > div.col-main
    contents = soup.select_one('body > div.wrap.imp-end > div.container.end-uni > div.end-body > div > div.col-main')
    content = ''
    p_tags = contents.select('p')
    for p in p_tags:
        content += p.text
    return link, content

In [132]:
link_to_get_content = {
    # 'biz.chosun.com': get_chosunbiz_content,
    'www.nocutnews.co.kr': get_nocut_content,
    'www.etnews.com': get_etnews_content,
    'www.hankyung.com': get_hankyung_content,
    'www.joongangilbo.co.kr': get_joongang_content,
    'news.mt.co.kr': get_moneytoday_content,
    'www.news1.kr': get_newsone_content,
    'www.hankookilbo.com': get_hankookilbo_content
}

In [133]:
def get_content(link):
    site_url = link.split('/')[2]
    get_content_func = link_to_get_content.get(site_url, None)
    if get_content_func:
        return True, get_content_func(link)
    else:
        return False, None

In [134]:
BASE_URL = r'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query='

comp_name = 'SK 하이닉스'

def get_url(comp_name):
    return BASE_URL + comp_name.replace(' ', '+')

In [135]:
get_url(comp_name)

'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=SK+하이닉스'

In [136]:
def get_news(comp_name):
    url = get_url(comp_name)
    r = requests.get(url)
    soup = bs(r.text, 'lxml')
    # #main_pack > section > div > div.group_news > ul
    news_list = soup.select('#main_pack > section > div > div.group_news > ul > li')
    
    print(news_list)
    
    news_dict = {
        'link': [],
        'content': []
    }
    
    for news in news_list:
        a_tag = news.select_one('div.news_wrap.api_ani_send > div > a')
        link = a_tag['href']
        print(link)
        is_success, data = get_content(link)
        if is_success:
            link, content = data
            news_dict['link'].append(link)
            news_dict['content'].append(content)
    
    return pd.DataFrame(news_dict)

In [137]:
get_news(comp_name)

[<li class="bx" id="sp_nws1"> <div class="news_wrap api_ani_send"> <div class="news_area"> <div class="news_info"> <div class="api_save_group _keep_wrap"> <a aria-pressed="false" class="btn_save _keep_trigger" data-url="https://biz.chosun.com/it-science/ict/2023/07/04/JZ2AOIDEI5DP7PBPVHR2RK2FFE/?utm_source=naver&amp;utm_medium=original&amp;utm_campaign=biz" href="#" onclick="tCR('a=nws*h.kep&amp;r=1&amp;i=88127058_000000000000000000913922&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=nws*h.kepoff&amp;r=1&amp;i=88127058_000000000000000000913922&amp;u=javascript" data-cr-on="a=nws*h.kepon&amp;r=1&amp;i=88127058_000000000000000000913922&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://keep.naver.com/" onclick="return goOtherCR(this,'a=nws*h.kephome&amp;r=1&amp;i=88127058_00000000000000000

,link,content
0,https://www.nocutnews.co.kr/news/5970546,\nSK하이닉스 제공SK하이닉스는 SK스퀘어가 효율적인 해외 반도체 투자를 위해 설...
1,https://www.etnews.com/20230704000192,SK가 일본 반도체 소재·부품·장비(소부장) 기술 확보에 나선다.\n\nSK하이닉스...
2,https://www.hankyung.com/finance/article/20230...,\n\n\n\n\n사진=한경DB\n이베스트투자증권은 5일 \n ...
3,http://news.mt.co.kr/mtview.php?no=20230704103...,\n\n\nSK하이닉스와 SK스퀘어가 국내 대표 금융사 등과 약 1000억원을 공동...
4,https://www.news1.kr/articles/5097016,\n\n\n\n\n\n\nê²½ê¸° ì´ì²ì SKíì´ëì¤ ...
5,https://www.hankookilbo.com/News/Read/A2023070...,"SK스퀘어-하이닉스, 1,000억 규모 투자법인 세워각국 정부 반도체 산업 지원 기..."


# 모듈화된 크롤러 테스트

In [2]:
from naver_news_crawler import get_news

get_news('네이버')

https://www.yna.co.kr/view/AKR20230705046600017?input=1195m
http://news.tvchosun.com/site/data/html_dir/2023/07/04/2023070490094.html
http://www.newsis.com/view/?id=NISX20230704_0002362846&cID=10301&pID=10300
http://www.mediatoday.co.kr/news/articleView.html?idxno=311078
http://news.mt.co.kr/mtview.php?no=2023070416093157795
https://www.etnews.com/20230705000054
http://www.newsis.com/view/?id=NISX20230704_0002363636&cID=13006&pID=13100
https://economist.co.kr/article/view/ecn202307040023
https://www.hankyung.com/it/article/202307043349i
https://zdnet.co.kr/view/?no=20230702214155
https://n.news.naver.com/mnews/article/003/0011954239?sid=105
https://n.news.naver.com/mnews/article/366/0000914245?sid=105
http://www.newspim.com/news/view/20230705000222
https://n.news.naver.com/mnews/article/014/0005037213?sid=105
https://n.news.naver.com/mnews/article/014/0005036623?sid=100
https://n.news.naver.com/mnews/article/421/0006905890?sid=100
https://n.news.naver.com/mnews/article/052/0001905199?s

,link,content
0,http://news.mt.co.kr/mtview.php?no=20230704160...,\n네이버웹툰이 프랑스 파리 생 라자르역에 게재한 빌보드 광고. /사진=네이버웹툰\...
1,https://www.etnews.com/20230705000054,파스토는 임종균 신임 최고기술책임자(CTO)와 이준성 신임 최고마케팅책임자(CMO)...
2,https://www.hankyung.com/it/article/202307043349i,\n\n\n\n\n‘K-컬처’의 대표주자인 웹툰이 한국 식품업체들의 동남아 홍보 창...
3,https://n.news.naver.com/mnews/article/003/001...,"\n원본 데이터 유추 불가능한 솔루션 개발""초거대 생성 AI 활용에 유용""\n\n\..."
4,https://n.news.naver.com/mnews/article/366/000...,\n\n\n\n\n네이버 제공 네이버 D2SF(D2 Startup Fa...
5,https://n.news.naver.com/mnews/article/014/000...,\n원본 데이터와 유사도 높으면서도 민감정보노출 위험성을 낮춘 데이터 생성 솔루션 ...
6,https://n.news.naver.com/mnews/article/014/000...,\n\n\n\n\n[서울=뉴시스] 고범준 기자 = 박성중 소위원장이 지난달 21일 ...
7,https://n.news.naver.com/mnews/article/421/000...,"\n""영향 미미하다니…자기 부정으로 용서되는 게 아냐""네이버 '자의적 선택' 어려워..."
8,https://n.news.naver.com/mnews/article/052/000...,\n\t\t\t국회 과학기술정보방송통신위원회 여당 간사를 맡고 있는 국민의힘 박성중...
9,https://n.news.naver.com/mnews/article/008/000...,\n[the300]\n\n\n\n(서울=뉴스1) 임세영 기자 = 박성중 국민의힘 과...


In [ ]:
'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={comp_name}sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=79&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={page_num}'